# 02 — Clean & Geo‑Join (Sensors → SA2)

In [ ]:
print('TODO')

In [3]:
import os, pathlib
print("cwd:", os.getcwd())
print("notebook file is in:", pathlib.Path(".").resolve())


cwd: /Users/poojithraj/Documents/melbourne-foot-traffic-marketing/notebooks
notebook file is in: /Users/poojithraj/Documents/melbourne-foot-traffic-marketing/notebooks


In [4]:
from pathlib import Path
raw = Path("../data/raw")   # <-- go up one level, then into data/raw
print("Looking in:", raw.resolve())
assert raw.exists(), "raw folder not found—path is wrong"
for p in sorted(raw.glob("*")):
    print(p.name, f"{p.stat().st_size/1e6:.2f} MB")


Looking in: /Users/poojithraj/Documents/melbourne-foot-traffic-marketing/data/raw
com_counts_2025_03.csv 5.04 MB
com_counts_2025_04.csv 4.11 MB
com_counts_2025_05.csv 4.94 MB
sensor_locations.geojson 0.00 MB


In [1]:
import re, glob
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd

# because this notebook is inside /notebooks, the project root is one level up
BASE = Path("..").resolve()
RAW = BASE / "data" / "raw"
INTERIM = BASE / "data" / "interim"
INTERIM.mkdir(parents=True, exist_ok=True)

MEL_TZ = "Australia/Melbourne"

def std_col(name: str) -> str:
    return re.sub(r"[^a-z0-9]+", "_", name.strip().lower()).strip("_")

def month_bounds_from_filename(p: Path):
    m = re.search(r"(\d{4})[_-](\d{2})", p.stem)
    if not m: return None, None
    y, mth = int(m.group(1)), int(m.group(2))
    start = pd.Timestamp(year=y, month=mth, day=1, tz=MEL_TZ)
    end   = (start + pd.offsets.MonthEnd(1)).replace(hour=23)
    return start, end

def filter_to_its_month(df: pd.DataFrame, src_file: Path) -> pd.DataFrame:
    start, end = month_bounds_from_filename(src_file)
    if start is None: return df
    mask = (df["date_time"] >= start) & (df["date_time"] <= end)
    return df.loc[mask].copy()


In [2]:
def coerce_counts_schema(df: pd.DataFrame) -> pd.DataFrame:
    original_cols = df.columns.tolist()
    df = df.rename(columns={c: std_col(c) for c in df.columns})

    # sensor id
    sid = None
    for c in ("sensor_id","location_id","sensorid","locationid"):
        if c in df.columns: sid = c; break
    assert sid, f"Could not find sensor id. Saw: {original_cols}"

    # counts
    cnt = None
    for c in ("hourly_counts","count","total_of_directions","total"):
        if c in df.columns: cnt = c; break
    assert cnt, f"Could not find counts column. Saw: {original_cols}"

    # timestamp (either a single column or date+hour)
    tscol = None
    for c in ("date_time","datetime","datehour"):
        if c in df.columns: tscol = c; break

    if tscol:
        s = pd.to_datetime(df[tscol], errors="coerce")
        if s.dt.tz is None:
            s = s.dt.tz_localize(MEL_TZ, ambiguous="NaT", nonexistent="NaT")
        else:
            s = s.dt.tz_convert(MEL_TZ)
        df["date_time"] = s
    else:
        dcol = next((c for c in ("sensing_date","date","day") if c in df.columns), None)
        hcol = next((c for c in ("hourday","hour","hr") if c in df.columns), None)
        assert dcol and hcol, f"Need date + hour. Saw: {original_cols}"
        dd = pd.to_datetime(df[dcol], errors="coerce")
        hh = pd.to_numeric(df[hcol], errors="coerce").astype("Int64")
        s  = dd + pd.to_timedelta(hh.astype(float), unit="h")
        s  = s.dt.tz_localize(MEL_TZ, ambiguous="NaT", nonexistent="NaT")
        df["date_time"] = s

    df["sensor_id"] = df[sid].astype(str).str.strip()
    df["hourly_counts"] = pd.to_numeric(df[cnt], errors="coerce")
    return df[["sensor_id","date_time","hourly_counts"]].copy()

# load & combine
csv_paths = sorted(RAW.glob("com_counts_*.csv"))
assert csv_paths, f"No files found in {RAW}"
frames = []
for p in csv_paths:
    t = pd.read_csv(p)
    t = coerce_counts_schema(t)
    t = t.dropna(subset=["sensor_id","date_time","hourly_counts"])
    t = filter_to_its_month(t, p)     # removes stray June/July rows from Apr/May files
    t["source_file"] = p.name
    frames.append(t)

counts = pd.concat(frames, ignore_index=True)

# de-dupe & validate
before = len(counts)
counts = counts.drop_duplicates(subset=["sensor_id","date_time"])
dups = before - len(counts)
neg  = (counts["hourly_counts"] < 0).sum()
assert neg == 0, f"Negative counts found: {neg}"
assert counts["date_time"].isna().sum() == 0, "Unparseable timestamps."

print(f"Rows: {len(counts):,} | Duplicates removed: {dups:,}")
print("Date range:", counts['date_time'].min(), "→", counts['date_time'].max())
print("Sensors:", counts['sensor_id'].nunique())
counts.head(3)


Rows: 64,040 | Duplicates removed: 0
Date range: 2025-03-01 00:00:00+11:00 → 2025-03-31 23:00:00+11:00
Sensors: 96


,sensor_id,date_time,hourly_counts,source_file
0,107,2025-03-01 18:00:00+11:00,237,com_counts_2025_03.csv
1,20,2025-03-01 14:00:00+11:00,602,com_counts_2025_03.csv
2,107,2025-03-15 02:00:00+11:00,22,com_counts_2025_03.csv


In [3]:
loc_geo = RAW / "sensor_locations.geojson"
assert loc_geo.exists(), f"{loc_geo} not found"

sensors = geopandas_read = gpd.read_file(loc_geo)
sensors = sensors.rename(columns={c: std_col(c) for c in sensors.columns})

required = {"location_id","sensor_name","sensor_description","installation_date"}
missing = required - set(sensors.columns)
assert not missing, f"Missing fields: {missing}. Present: {list(sensors.columns)}"

latcol = "latitude" if "latitude" in sensors.columns else "lat"
loncol = "longitude" if "longitude" in sensors.columns else "lon"
assert latcol in sensors.columns and loncol in sensors.columns, "latitude/longitude not found."

sensors_clean = sensors.copy()
sensors_clean["sensor_id"]  = sensors_clean["location_id"].astype(str).str.strip()
sensors_clean["latitude"]   = pd.to_numeric(sensors_clean[latcol], errors="coerce")
sensors_clean["longitude"]  = pd.to_numeric(sensors_clean[loncol], errors="coerce")
assert sensors_clean["latitude"].notna().all() and sensors_clean["longitude"].notna().all(), "Some sensors missing coords."

sensors_clean = sensors_clean[["sensor_id","sensor_name","sensor_description","installation_date","latitude","longitude"]]
sensors_clean = sensors_clean.drop_duplicates(subset=["sensor_id"])

print("Sensors table shape:", sensors_clean.shape)
sensors_clean.head(3)


Sensors table shape: (5, 6)


,sensor_id,sensor_name,sensor_description,installation_date,latitude,longitude
0,181,Eli368_T,368 Elizabeth Street,2025-03-26,-37.810095,144.961431
1,184,Eli124_T,124 Elizabeth Street,2025-06-28,-37.815124,144.963720
3,185,Eli197_T,197 Elizabeth Street,2025-06-28,-37.813746,144.962762


In [4]:
# coverage
cnt_ids = set(counts["sensor_id"].unique())
loc_ids = set(sensors_clean["sensor_id"].unique())
missing = cnt_ids - loc_ids
coverage = 100 * (1 - len(missing)/max(1,len(cnt_ids)))
print(f"Sensor ID coverage: {coverage:.1f}% (missing {len(missing)})")
if missing:
    print("Example missing IDs:", list(sorted(missing))[:10])

# save
counts_out  = INTERIM / "traffic_by_hour.csv"
sensors_out = INTERIM / "sensor_locations_clean.csv"
counts[["sensor_id","date_time","hourly_counts"]].sort_values(["sensor_id","date_time"]).to_csv(counts_out, index=False)
sensors_clean.to_csv(sensors_out, index=False)
print("Wrote:", counts_out)
print("Wrote:", sensors_out)


Sensor ID coverage: 2.1% (missing 94)
Example missing IDs: ['1', '10', '107', '108', '109', '11', '117', '118', '12', '123']
Wrote: /Users/poojithraj/Documents/melbourne-foot-traffic-marketing/data/interim/traffic_by_hour.csv
Wrote: /Users/poojithraj/Documents/melbourne-foot-traffic-marketing/data/interim/sensor_locations_clean.csv


In [5]:
from pathlib import Path
p = Path("../data/raw/sensor_locations.geojson")
print("Exists:", p.exists(), "| Size MB:", round(p.stat().st_size/1e6, 2))


Exists: True | Size MB: 0.05


In [6]:
from pathlib import Path
p = Path("../data/raw/sensor_locations.geojson")
print("Exists:", p.exists(), "| Size MB:", round(p.stat().st_size/1e6, 2))


Exists: True | Size MB: 0.05


In [7]:
import geopandas as gpd, pandas as pd, re
from pathlib import Path

loc_geo = Path("../data/raw/sensor_locations.geojson")
sensors = gpd.read_file(loc_geo).rename(columns=lambda c: re.sub(r"[^a-z0-9]+","_", c.lower()).strip("_"))

required = {"location_id","sensor_name","sensor_description","installation_date"}
missing = required - set(sensors.columns)
assert not missing, f"Missing fields: {missing}. Present: {list(sensors.columns)}"

latcol = "latitude" if "latitude" in sensors.columns else "lat"
loncol = "longitude" if "longitude" in sensors.columns else "lon"
assert latcol in sensors.columns and loncol in sensors.columns, "latitude/longitude not found."

sensors_clean = sensors.copy()
sensors_clean["sensor_id"]  = sensors_clean["location_id"].astype(str).str.strip()
sensors_clean["latitude"]   = pd.to_numeric(sensors_clean[latcol], errors="coerce")
sensors_clean["longitude"]  = pd.to_numeric(sensors_clean[loncol], errors="coerce")
assert sensors_clean["latitude"].notna().all() and sensors_clean["longitude"].notna().all()

sensors_clean = sensors_clean[["sensor_id","sensor_name","sensor_description","installation_date","latitude","longitude"]]
sensors_clean = sensors_clean.drop_duplicates(subset=["sensor_id"])

print("Sensors table shape:", sensors_clean.shape)
sensors_clean.head(3)


Sensors table shape: (135, 6)


,sensor_id,sensor_name,sensor_description,installation_date,latitude,longitude
0,1,Bou292_T,Bourke Street Mall (North),2009-03-24,-37.813494,144.965153
1,4,Swa123_T,Town Hall (West),2009-03-23,-37.814880,144.966088
2,10,BouHbr_T,Victoria Point,2009-04-23,-37.818765,144.947105


In [8]:
cnt_ids = set(counts["sensor_id"].unique())
loc_ids = set(sensors_clean["sensor_id"].unique())
missing = cnt_ids - loc_ids
coverage = 100 * (1 - len(missing)/max(1,len(cnt_ids)))
print(f"Sensor ID coverage: {coverage:.1f}% (missing {len(missing)})")
if missing:
    print("Example missing IDs:", list(sorted(missing))[:10])


Sensor ID coverage: 100.0% (missing 0)


In [9]:
from pathlib import Path
INTERIM = Path("..")/"data"/"interim"
INTERIM.mkdir(parents=True, exist_ok=True)
counts[["sensor_id","date_time","hourly_counts"]].sort_values(["sensor_id","date_time"]).to_csv(INTERIM/"traffic_by_hour.csv", index=False)
sensors_clean.to_csv(INTERIM/"sensor_locations_clean.csv", index=False)
print("Wrote:", INTERIM/"traffic_by_hour.csv")
print("Wrote:", INTERIM/"sensor_locations_clean.csv")


Wrote: ../data/interim/traffic_by_hour.csv
Wrote: ../data/interim/sensor_locations_clean.csv


In [10]:
git add data/interim/*.csv notebooks/02_clean_join_geo.ipynb
git commit -m "Use full sensor locations; coverage ok; save clean tables"
git push


SyntaxError: invalid decimal literal (970244286.py, line 1)

In [11]:
# sanity checks (prints repo status, remote and branch)
!git -C .. rev-parse --is-inside-work-tree
!git -C .. status
!git -C .. remote -v
!git -C .. branch

# stage the notebook you edited (DO NOT add data/interim)
!git -C .. add notebooks/02_clean_join_geo.ipynb

# if you already created KPI CSVs earlier, stage that folder too (safe even if empty)
!git -C .. add -A analytics/looker_studio_datasources

# commit
!git -C .. commit -m "Clean CoM hourly counts + full sensor locations (coverage 100%); save interim outputs"

# push to GitHub (main branch)
!git -C .. push -u origin main


true
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    notebooks/01_download_coM.ipynb
	modified:   notebooks/02_clean_join_geo.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Miniforge3-MacOSX-arm64.sh
	notebooks/01_download_coM_FIXED.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
origin	https://github.com/RajPoo7/melbourne-foot-traffic-marketing.git (fetch)
origin	https://github.com/RajPoo7/melbourne-foot-traffic-marketing.git (push)
* main
fatal: pathspec 'analytics/looker_studio_datasources' did not match any files
[main 8dd3a7a] Clean CoM hourly counts + full sensor locations (coverage 100%); save interim outputs
 1 file changed, 657 insertions(+), 1 deletion(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done

In [12]:
import pandas as pd
from pathlib import Path

# Load clean hourly data
df = pd.read_csv("../data/interim/traffic_by_hour.csv", parse_dates=["date_time"])
df["weekday"] = df["date_time"].dt.day_name()
df["hour"]    = df["date_time"].dt.hour

# 1) Weekday × hour heatmap
weekday_order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
heatmap = (df.groupby(["weekday","hour"])["hourly_counts"]
             .mean()
             .reset_index()
             .rename(columns={"hourly_counts":"avg_count"}))
heatmap["weekday"] = pd.Categorical(heatmap["weekday"], categories=weekday_order, ordered=True)
heatmap = heatmap.sort_values(["weekday","hour"])

# 2) Top 5 "power hours" per sensor
power = (df.groupby(["sensor_id","hour"])["hourly_counts"]
           .mean()
           .reset_index()
           .rename(columns={"hourly_counts":"avg_count"}))
power["rank_in_sensor"] = power.groupby("sensor_id")["avg_count"].rank(method="first", ascending=False)
power_hours = power.query("rank_in_sensor <= 5").sort_values(["sensor_id","rank_in_sensor"])

# Save to a git-tracked folder
outdir = Path("../analytics/looker_studio_datasources")
outdir.mkdir(parents=True, exist_ok=True)
heatmap.to_csv(outdir/"heatmap.csv", index=False)
power_hours.to_csv(outdir/"power_hours.csv", index=False)
print("Wrote:", outdir/"heatmap.csv")
print("Wrote:", outdir/"power_hours.csv")


Wrote: ../analytics/looker_studio_datasources/heatmap.csv
Wrote: ../analytics/looker_studio_datasources/power_hours.csv


In [13]:
# stage the two CSVs and your notebook
!git -C .. add analytics/looker_studio_datasources/*.csv notebooks/02_clean_join_geo.ipynb

# commit & push
!git -C .. commit -m "Export heatmap & power-hours for dashboard"
!git -C .. push


zsh:1: no matches found: analytics/looker_studio_datasources/*.csv
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    notebooks/01_download_coM.ipynb
	modified:   notebooks/02_clean_join_geo.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Miniforge3-MacOSX-arm64.sh
	analytics/
	notebooks/01_download_coM_FIXED.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
Everything up-to-date


In [14]:
!git -C .. add notebooks/01_download_coM.ipynb notebooks/01_download_coM_FIXED.ipynb
!git -C .. commit -m "Keep both 01_* notebooks (original + FIXED)"
!git -C .. push


[main 1eb8475] Keep both 01_* notebooks (original + FIXED)
 2 files changed, 178 insertions(+), 25 deletions(-)
 delete mode 100644 notebooks/01_download_coM.ipynb
 create mode 100644 notebooks/01_download_coM_FIXED.ipynb
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 3.37 KiB | 3.37 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/RajPoo7/melbourne-foot-traffic-marketing.git
   8dd3a7a..1eb8475  main -> main


In [15]:
# Remove the old file from the repo, keep FIXED
!git -C .. rm --cached notebooks/01_download_coM.ipynb  # stops tracking but leaves your local copy
!git -C .. add notebooks/01_download_coM_FIXED.ipynb
!git -C .. commit -m "Replace 01_download_coM with 01_download_coM_FIXED"
!git -C .. push


fatal: pathspec 'notebooks/01_download_coM.ipynb' did not match any files
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   notebooks/02_clean_join_geo.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Miniforge3-MacOSX-arm64.sh
	analytics/

no changes added to commit (use "git add" and/or "git commit -a")
Everything up-to-date


In [16]:
import pandas as pd
from pathlib import Path

# Load clean hourly series
df = pd.read_csv("../data/interim/traffic_by_hour.csv", parse_dates=["date_time"])
df["weekday"] = df["date_time"].dt.day_name()
df["hour"]    = df["date_time"].dt.hour

# 1) Weekday × hour heatmap
weekday_order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
heatmap = (df.groupby(["weekday","hour"])["hourly_counts"]
             .mean()
             .reset_index()
             .rename(columns={"hourly_counts":"avg_count"}))
heatmap["weekday"] = pd.Categorical(heatmap["weekday"], categories=weekday_order, ordered=True)
heatmap = heatmap.sort_values(["weekday","hour"])

# 2) Top 5 “power hours” per sensor
power = (df.groupby(["sensor_id","hour"])["hourly_counts"]
           .mean()
           .reset_index()
           .rename(columns={"hourly_counts":"avg_count"}))
power["rank_in_sensor"] = power.groupby("sensor_id")["avg_count"].rank(method="first", ascending=False)
power_hours = power.query("rank_in_sensor <= 5").sort_values(["sensor_id","rank_in_sensor"])

# Save (this folder is tracked by git)
outdir = Path("../analytics/looker_studio_datasources")
outdir.mkdir(parents=True, exist_ok=True)
heatmap.to_csv(outdir/"heatmap.csv", index=False)
power_hours.to_csv(outdir/"power_hours.csv", index=False)
print("Wrote:", outdir/"heatmap.csv")
print("Wrote:", outdir/"power_hours.csv")


Wrote: ../analytics/looker_studio_datasources/heatmap.csv
Wrote: ../analytics/looker_studio_datasources/power_hours.csv


In [17]:
# Stage the two CSVs explicitly (avoids the wildcard issue)
!git -C .. add analytics/looker_studio_datasources/heatmap.csv analytics/looker_studio_datasources/power_hours.csv

# Stage the notebook you edited
!git -C .. add notebooks/02_clean_join_geo.ipynb

# Commit & push
!git -C .. commit -m "Export heatmap & power-hours for dashboard"
!git -C .. push


[main c0cb15f] Export heatmap & power-hours for dashboard
 3 files changed, 909 insertions(+), 2 deletions(-)
 create mode 100644 analytics/looker_studio_datasources/heatmap.csv
 create mode 100644 analytics/looker_studio_datasources/power_hours.csv
Enumerating objects: 11, done.
Counting objects: 100% (11/11), done.
Delta compression using up to 8 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 8.73 KiB | 4.36 MiB/s, done.
Total 8 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/RajPoo7/melbourne-foot-traffic-marketing.git
   1eb8475..c0cb15f  main -> main


In [18]:
# --- Sensors -> SA2 join (ASGS 2021, GDA2020) ---

import pandas as pd, geopandas as gpd
from pathlib import Path

# 1) load clean sensors (we already built this earlier)
sensors = pd.read_csv("../data/interim/sensor_locations_clean.csv")
assert {"sensor_id","latitude","longitude"}.issubset(sensors.columns), "sensor_locations_clean.csv missing columns"

# 2) find the SA2 shapefile we just unzipped
sa2_dir = Path("../data/raw/SA2")
shp = next(sa2_dir.glob("*.shp"), None)
assert shp is not None, "Put the SA2 2021 shapefile files into data/raw/SA2/"

# 3) build GeoDataFrames (sensors in EPSG:4326; SA2 is GDA2020 geographic)
gdf_s = gpd.GeoDataFrame(
    sensors.copy(),
    geometry=gpd.points_from_xy(sensors["longitude"], sensors["latitude"]),
    crs="EPSG:4326"  # WGS84 lon/lat
)

gdf_sa2 = gpd.read_file(shp)       # ABS ASGS 2021 SA2 (GDA2020)
# To get precise, consistent spatial math in VIC, use GDA2020 / MGA Zone 55 (EPSG:7855)
gdf_s   = gdf_s.to_crs(epsg=7855)
gdf_sa2 = gdf_sa2.to_crs(epsg=7855)

# 4) detect the SA2 name column (ABS sometimes labels as SA2_NAME_2021)
sa2_name_col = next((c for c in gdf_sa2.columns if "SA2_NAME" in c.upper()), None)
assert sa2_name_col, f"Could not find SA2 name column in {list(gdf_sa2.columns)[:8]}..."

# 5) spatial join: which SA2 polygon contains each sensor point?
joined = gpd.sjoin(
    gdf_s,
    gdf_sa2[[sa2_name_col, "geometry"]],
    how="left",
    predicate="within"
).drop(columns=["index_right"])

out = joined[["sensor_id","sensor_name", sa2_name_col]].rename(columns={sa2_name_col:"SA2_NAME"})
out.to_csv("../data/interim/sensor_sa2.csv", index=False)
print("Wrote: ../data/interim/sensor_sa2.csv")
out.head()


Wrote: ../data/interim/sensor_sa2.csv


,sensor_id,sensor_name,SA2_NAME
0,1,Bou292_T,Melbourne CBD - East
1,4,Swa123_T,Melbourne CBD - East
2,10,BouHbr_T,Docklands
3,11,WatCit_T,Docklands
4,24,Col620_T,Melbourne CBD - West


In [19]:
# stage the updated notebook (and keep code tidy in repo)
!git -C .. add notebooks/02_clean_join_geo.ipynb
!git -C .. commit -m "Add SA2 (ASGS 2021 GDA2020) join: sensor -> SA2_NAME"
!git -C .. push


[main 3ea8340] Add SA2 (ASGS 2021 GDA2020) join: sensor -> SA2_NAME
 1 file changed, 57 insertions(+), 2 deletions(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 800 bytes | 800.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/RajPoo7/melbourne-foot-traffic-marketing.git
   c0cb15f..3ea8340  main -> main


In [1]:
import pandas as pd; pd.read_csv("../data/interim/sensor_sa2.csv").head()


,sensor_id,sensor_name,SA2_NAME
0,1,Bou292_T,Melbourne CBD - East
1,4,Swa123_T,Melbourne CBD - East
2,10,BouHbr_T,Docklands
3,11,WatCit_T,Docklands
4,24,Col620_T,Melbourne CBD - West


In [2]:
import pandas as pd, re
from pathlib import Path

# 1) Locate the ABS workbook you saved
src = Path("../data/raw/ABS")
xlsx = (next(src.glob("abs_*2024*.xlsx"), None) or
        next(src.glob("abs_populat*2024*.xlsx"), None) or
        next(src.glob("*.xlsx"), None))
assert xlsx is not None, "Put the ABS 'Population and people' XLSX into data/raw/ABS/"
print("Using workbook:", xlsx.name)

# 2) Inspect sheets, pick one with SA2 in the name if available
xl = pd.ExcelFile(xlsx)
cand = [s for s in xl.sheet_names if "SA2" in s.upper()]
sheet = cand[0] if cand else xl.sheet_names[0]
print("Reading sheet:", sheet)
df0 = xl.parse(sheet)
print("Columns sample:", list(df0.columns)[:8])

# 3) Try to detect "long" vs "wide" structure
lower = {c.lower(): c for c in df0.columns}

# likely column names across releases
name_candidates = [c for c in df0.columns if ("SA2" in c.upper() and "NAME" in c.upper()) 
                   or ("region" in c.lower() and "name" in c.lower())]
name_col = name_candidates[0] if name_candidates else None

item_col = next((c for c in df0.columns if "data item" in c.lower() or "indicator" in c.lower()), None)
time_col = next((c for c in df0.columns if c.lower() in ["time","year"]), None)
value_col = next((c for c in df0.columns if "value" in c.lower()), None)

def build_from_long(df):
    # Filter to Estimated resident population and latest year
    pop = df[df[item_col].astype(str).str.contains("Estimated resident population", case=False, na=False)].copy()
    latest = pop[time_col].max()
    pop = pop[pop[time_col] == latest]
    use_name = name_col or next(c for c in pop.columns if "name" in c.lower())
    pop = pop[[use_name, value_col]].rename(columns={use_name:"SA2_NAME", value_col:"SA2_POP"})
    return pop

def build_from_wide(df):
    # Find the right-most ERP/year column
    use_name = name_col or next(c for c in df.columns if "name" in c.lower())
    year_cols = [c for c in df.columns if re.search(r"(ERP|resident population).*20\d{2}", str(c), flags=re.I)]
    if year_cols:
        # pick latest year by number in the header
        def yearnum(c):
            m = re.findall(r"(20\d{2})", str(c))
            return int(m[-1]) if m else 0
        latest_col = max(year_cols, key=yearnum)
    else:
        # fallback: any column literally named ERP or Population
        latest_col = next(c for c in df.columns if re.search(r"ERP|population", str(c), flags=re.I))
    pop = df[[use_name, latest_col]].rename(columns={use_name:"SA2_NAME", latest_col:"SA2_POP"})
    return pop

if item_col and time_col and value_col:
    pop_tidy = build_from_long(df0)
else:
    pop_tidy = build_from_wide(df0)

# Tidy types
pop_tidy["SA2_NAME"] = pop_tidy["SA2_NAME"].astype(str).str.strip()
pop_tidy["SA2_POP"]  = pd.to_numeric(pop_tidy["SA2_POP"], errors="coerce")

# 4) Load your SA2 mapping (from Step 1) and join
sa2_map = pd.read_csv("../data/interim/sensor_sa2.csv")
sa2_map["SA2_NAME"] = sa2_map["SA2_NAME"].astype(str).str.strip()

sensor_demo = sa2_map.merge(pop_tidy, on="SA2_NAME", how="left")
out = "../data/interim/sensor_sa2_population.csv"
sensor_demo.to_csv(out, index=False)
print("Wrote:", out)
sensor_demo.head()


Using workbook: abs_population_people_2011_2024.xlsx


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [2]:
import sys
!{sys.executable} -m pip install --quiet openpyxl


In [3]:
import sys
!{sys.executable} -m pip install --upgrade --quiet openpyxl


In [4]:
import openpyxl, pandas as pd
print("openpyxl:", openpyxl.__version__)
print("pandas:", pd.__version__)


openpyxl: 3.1.5
pandas: 2.3.2


In [1]:
from pathlib import Path
import pandas as pd

# point to your ABS workbook (the one you downloaded)
xlsx = next(Path("../data/raw/ABS").glob("*.xlsx"))
print("Using workbook:", xlsx.name)

xl = pd.ExcelFile(xlsx, engine="openpyxl")   # <— explicit engine
print("Sheets:", xl.sheet_names)


Using workbook: abs_population_people_2011_2024.xlsx
Sheets: ['Contents', 'Table 1', 'Table 2', 'Table 3']


In [2]:
import pandas as pd, re
from pathlib import Path

# 1) Locate the ABS workbook you saved
src = Path("../data/raw/ABS")
xlsx = (next(src.glob("abs_*2024*.xlsx"), None) or
        next(src.glob("abs_populat*2024*.xlsx"), None) or
        next(src.glob("*.xlsx"), None))
assert xlsx is not None, "Put the ABS 'Population and people' XLSX into data/raw/ABS/"
print("Using workbook:", xlsx.name)

# 2) Inspect sheets, pick one with SA2 in the name if available
xl = pd.ExcelFile(xlsx)
cand = [s for s in xl.sheet_names if "SA2" in s.upper()]
sheet = cand[0] if cand else xl.sheet_names[0]
print("Reading sheet:", sheet)
df0 = xl.parse(sheet)
print("Columns sample:", list(df0.columns)[:8])

# 3) Try to detect "long" vs "wide" structure
lower = {c.lower(): c for c in df0.columns}

# likely column names across releases
name_candidates = [c for c in df0.columns if ("SA2" in c.upper() and "NAME" in c.upper()) 
                   or ("region" in c.lower() and "name" in c.lower())]
name_col = name_candidates[0] if name_candidates else None

item_col = next((c for c in df0.columns if "data item" in c.lower() or "indicator" in c.lower()), None)
time_col = next((c for c in df0.columns if c.lower() in ["time","year"]), None)
value_col = next((c for c in df0.columns if "value" in c.lower()), None)

def build_from_long(df):
    # Filter to Estimated resident population and latest year
    pop = df[df[item_col].astype(str).str.contains("Estimated resident population", case=False, na=False)].copy()
    latest = pop[time_col].max()
    pop = pop[pop[time_col] == latest]
    use_name = name_col or next(c for c in pop.columns if "name" in c.lower())
    pop = pop[[use_name, value_col]].rename(columns={use_name:"SA2_NAME", value_col:"SA2_POP"})
    return pop

def build_from_wide(df):
    # Find the right-most ERP/year column
    use_name = name_col or next(c for c in df.columns if "name" in c.lower())
    year_cols = [c for c in df.columns if re.search(r"(ERP|resident population).*20\d{2}", str(c), flags=re.I)]
    if year_cols:
        # pick latest year by number in the header
        def yearnum(c):
            m = re.findall(r"(20\d{2})", str(c))
            return int(m[-1]) if m else 0
        latest_col = max(year_cols, key=yearnum)
    else:
        # fallback: any column literally named ERP or Population
        latest_col = next(c for c in df.columns if re.search(r"ERP|population", str(c), flags=re.I))
    pop = df[[use_name, latest_col]].rename(columns={use_name:"SA2_NAME", latest_col:"SA2_POP"})
    return pop

if item_col and time_col and value_col:
    pop_tidy = build_from_long(df0)
else:
    pop_tidy = build_from_wide(df0)

# Tidy types
pop_tidy["SA2_NAME"] = pop_tidy["SA2_NAME"].astype(str).str.strip()
pop_tidy["SA2_POP"]  = pd.to_numeric(pop_tidy["SA2_POP"], errors="coerce")

# 4) Load your SA2 mapping (from Step 1) and join
sa2_map = pd.read_csv("../data/interim/sensor_sa2.csv")
sa2_map["SA2_NAME"] = sa2_map["SA2_NAME"].astype(str).str.strip()

sensor_demo = sa2_map.merge(pop_tidy, on="SA2_NAME", how="left")
out = "../data/interim/sensor_sa2_population.csv"
sensor_demo.to_csv(out, index=False)
print("Wrote:", out)
sensor_demo.head()


Using workbook: abs_population_people_2011_2024.xlsx
Reading sheet: Contents
Columns sample: ['            Australian Bureau of Statistics', 'Unnamed: 1', 'Unnamed: 2']


StopIteration: 

In [3]:
import pandas as pd, re
from pathlib import Path

# 1) find your ABS workbook (.xlsx)
xlsx = next(Path("../data/raw/ABS").glob("*.xlsx"))
print("Using workbook:", xlsx.name)

# 2) open with openpyxl and discover the correct sheet
xl = pd.ExcelFile(xlsx, engine="openpyxl")
print("Sheets:", xl.sheet_names)

sheet_chosen, df0 = None, None
for s in xl.sheet_names:
    df = xl.parse(s, dtype=str)   # read as strings to be robust
    cols_norm = [str(c).strip() for c in df.columns]
    lc = [c.lower() for c in cols_norm]
    # look for a tidy "long" table: Region type | Region name | Data item | Time | Value
    if (any("region type" in c for c in lc) and
        any(("region name" in c) or (c == "region") for c in lc) and
        any("data item" in c for c in lc) and
        any((c in ["time", "year"]) or ("time" in c) for c in lc) and
        any("value" in c for c in lc)):
        sheet_chosen, df0 = s, df
        break

assert df0 is not None, f"Could not find a tidy sheet in {xl.sheet_names}"
print("Chosen sheet:", sheet_chosen)
print("Columns:", list(df0.columns)[:8])

# 3) standardise column names
colmap = {}
for c in df0.columns:
    cl = str(c).lower()
    if "region type" in cl:            colmap[c] = "region_type"
    elif ("region name" in cl) or cl=="region": colmap[c] = "region_name"
    elif "data item" in cl:            colmap[c] = "data_item"
    elif cl in ("time","year") or " time" in cl: colmap[c] = "time"
    elif "value" in cl:                colmap[c] = "value"

df = df0.rename(columns=colmap)

# 4) keep SA2 rows only
sa2 = df[df["region_type"].str.contains("SA2", case=False, na=False)].copy()

# 5) keep the Estimated resident population rows
erp = sa2[sa2["data_item"].str.contains("Estimated resident population", case=False, na=False)].copy()

# 6) pick the latest year (handles values like '2023' or '2023-24')
def to_year(s):
    m = re.findall(r"20\d{2}", str(s))
    return int(m[-1]) if m else None

erp["year"] = erp["time"].apply(to_year)
latest_year = erp["year"].max()
erp_latest = erp[erp["year"] == latest_year].copy()

# 7) clean columns for join
erp_latest["SA2_NAME"] = erp_latest["region_name"].astype(str).str.strip()
erp_latest["SA2_POP"]  = pd.to_numeric(erp_latest["value"], errors="coerce")

print("Latest year detected:", latest_year)
erp_latest.head()


Using workbook: abs_population_people_2011_2024.xlsx
Sheets: ['Contents', 'Table 1', 'Table 2', 'Table 3']


AssertionError: Could not find a tidy sheet in ['Contents', 'Table 1', 'Table 2', 'Table 3']

In [4]:
import pandas as pd, re
from pathlib import Path

# 1) locate your ABS workbook
xlsx = next(Path("../data/raw/ABS").glob("*.xlsx"))
print("Using workbook:", xlsx.name)

xl = pd.ExcelFile(xlsx, engine="openpyxl")
print("Sheets:", xl.sheet_names)

def read_abs_long_sheet(xl, sheet):
    """Read an ABS Data-by-Region sheet where the header row is NOT the first row.
       Returns a tidy df with columns: region_type, region_name, data_item, time, value
       or None if this sheet doesn't match the pattern."""
    raw = xl.parse(sheet, header=None, dtype=str)

    header_row = None
    cols = None

    # Look for the header row in the first ~50 lines
    for i in range(min(50, len(raw))):
        row = [str(x).strip() for x in raw.iloc[i].tolist()]
        low = " ".join([c.lower() for c in row])
        if ("region type" in low and
            ("region name" in low or " region " in low or "region" in low) and
            "data item" in low and
            (" time" in low or "time " in low or "year" in low) and
            "value" in low):
            header_row = i
            cols = row
            break

    if header_row is None:
        return None

    df = raw.iloc[header_row+1:].copy()
    df.columns = cols
    df = df.dropna(how="all")

    # Standardise names
    colmap = {}
    for c in df.columns:
        cl = str(c).lower()
        if "region type" in cl:                   colmap[c] = "region_type"
        elif "region name" in cl or cl=="region": colmap[c] = "region_name"
        elif "data item" in cl:                   colmap[c] = "data_item"
        elif cl=="time" or "year" in cl:          colmap[c] = "time"
        elif "value" in cl:                       colmap[c] = "value"

    df = df.rename(columns=colmap)

    needed = {"region_type","region_name","data_item","time","value"}
    if not needed.issubset(df.columns):
        return None

    return df

# try each sheet until one matches the tidy pattern
df0, sheet_chosen = None, None
for s in xl.sheet_names:
    attempt = read_abs_long_sheet(xl, s)
    if attempt is not None:
        df0, sheet_chosen = attempt, s
        break

assert df0 is not None, f"Couldn't find a tidy table in any sheet: {xl.sheet_names}"
print("Chosen sheet:", sheet_chosen)
print("Columns:", list(df0.columns))
df0.head()


Using workbook: abs_population_people_2011_2024.xlsx
Sheets: ['Contents', 'Table 1', 'Table 2', 'Table 3']


AssertionError: Couldn't find a tidy table in any sheet: ['Contents', 'Table 1', 'Table 2', 'Table 3']

In [5]:
from pathlib import Path
import pandas as pd, re

# 1) Point to your ABS workbook
xlsx = next(Path("../data/raw/ABS").glob("*.xlsx"))
print("Using workbook:", xlsx.name)

# 2) Open it and pick Table 1 (ASGS). If needed we’ll try the other tables.
xl = pd.ExcelFile(xlsx, engine="openpyxl")
print("Sheets:", xl.sheet_names)

def find_header_row(raw):
    """
    Scan the first ~200 rows to find the header line that contains:
    - region… (at least once)
    - data item
    - time/year
    - value (ABS sheets often use 'Value' or 'Value (number)')
    Returns (row_index, header_strings) or (None, None).
    """
    N = min(200, len(raw))
    for i in range(N):
        row = [str(x) if x is not None else "" for x in raw.iloc[i].tolist()]
        low = [c.strip().lower() for c in row]
        has_region = any("region" in c for c in low)
        has_item   = any("data item" in c for c in low)
        has_time   = any(("time" in c) or ("year" in c) for c in low)
        has_value  = any(c.startswith("value") for c in low)
        # require 3/4 signals (region + item + time/year + value)
        if (has_region + has_item + has_time + has_value) >= 3:
            return i, row
    return None, None

def read_tidy_from_sheet(xl, sheet_name):
    raw = xl.parse(sheet_name, header=None, dtype=str)
    hdr_idx, hdr = find_header_row(raw)
    if hdr_idx is None: 
        return None, None

    df = raw.iloc[hdr_idx+1:].copy()
    df.columns = hdr
    df = df.dropna(how="all")

    # Standardise likely column names
    colmap = {}
    for c in df.columns:
        cl = str(c).strip().lower()
        if "region type" in cl:                  colmap[c] = "region_type"
        elif ("region name" in cl) or cl=="region": colmap[c] = "region_name"
        elif "data item" in cl:                  colmap[c] = "data_item"
        elif (cl=="time") or ("year" in cl):     colmap[c] = "time"
        elif cl.startswith("value"):             colmap[c] = "value"
    df = df.rename(columns=colmap)

    needed = {"region_type","region_name","data_item","time","value"}
    if not needed.issubset(df.columns):
        return None, None
    return df, hdr_idx

# Try Table 1 first (ASGS). If not tidy, try the others.
order = ["Table 1", "Table 2", "Table 3", "Contents"]
df0, used_sheet = None, None
for s in order:
    if s in xl.sheet_names:
        df0, hdr_idx = read_tidy_from_sheet(xl, s)
        if df0 is not None:
            used_sheet = s
            break

assert df0 is not None, f"Couldn't find a tidy table — try re-downloading the ABS file. Sheets: {xl.sheet_names}"
print(f"Chosen sheet: {used_sheet} (header at row {hdr_idx})")
print("Columns:", list(df0.columns))
df0.head()


Using workbook: abs_population_people_2011_2024.xlsx
Sheets: ['Contents', 'Table 1', 'Table 2', 'Table 3']


AssertionError: Couldn't find a tidy table — try re-downloading the ABS file. Sheets: ['Contents', 'Table 1', 'Table 2', 'Table 3']

In [6]:
from pathlib import Path
import pandas as pd, re

# 1) point to the workbook you downloaded earlier
xlsx = next(Path("../data/raw/ABS").glob("*.xlsx"))
print("Using workbook:", xlsx.name)

xl = pd.ExcelFile(xlsx, engine="openpyxl")
print("Sheets:", xl.sheet_names)

def _norm_row(values):
    return [(str(x) if x is not None else "").strip() for x in values]

def _key_flags(row):
    low = [c.lower() for c in row]
    return {
        "region": any("region" in c for c in low),
        "item":   any("data item" in c for c in low),
        "time":   any(("time" in c) or ("year" in c) for c in low),
        "value":  any(c.startswith("value") or " value" in c for c in low),
    }

def _merge_two_rows(r1, r2):
    m = max(len(r1), len(r2))
    r1 += [""]*(m - len(r1))
    r2 += [""]*(m - len(r2))
    merged = []
    for a,b in zip(r1, r2):
        a,b = a.strip(), b.strip()
        if a and b and a.lower()!=b.lower():
            merged.append((a + " " + b).strip())
        else:
            merged.append(a or b)
    return merged

def _read_tidy_from_sheet(xl, sheet):
    raw = xl.parse(sheet, header=None, dtype=str)
    n = len(raw)

    # --- try single header row first
    for i in range(min(200, n)):
        row = _norm_row(raw.iloc[i].tolist())
        k = _key_flags(row)
        if (k["region"] + k["item"] + k["time"] + k["value"]) >= 3 and k["region"] and k["value"]:
            hdr = row
            df = raw.iloc[i+1:].copy()
            df.columns = hdr
            df = df.dropna(how="all")
            break
    else:
        # --- try two consecutive rows as header (common ABS pattern)
        found = False
        for i in range(min(200, n-1)):
            r1 = _norm_row(raw.iloc[i].tolist())
            r2 = _norm_row(raw.iloc[i+1].tolist())
            k1, k2 = _key_flags(r1), _key_flags(r2)
            k = {t: (k1[t] or k2[t]) for t in ("region","item","time","value")}
            if (k["region"] + k["item"] + k["time"] + k["value"]) >= 3 and k["region"] and k["value"]:
                hdr = _merge_two_rows(r1, r2)
                df  = raw.iloc[i+2:].copy()
                df.columns = hdr
                df = df.dropna(how="all")
                found = True
                break
        if not found:
            return None, None

    # standardise column names we need
    colmap = {}
    for c in df.columns:
        cl = str(c).lower()
        if "region type" in cl:                   colmap[c] = "region_type"
        elif ("region name" in cl) or cl=="region": colmap[c] = "region_name"
        elif "data item" in cl:                   colmap[c] = "data_item"
        elif cl=="time" or "year" in cl:          colmap[c] = "time"
        elif cl.startswith("value"):              colmap[c] = "value"
    df = df.rename(columns=colmap)

    need = {"region_type","region_name","data_item","time","value"}
    if not need.issubset(df.columns):
        return None, None

    return df, sheet

# Prefer Table 1 (ASGS); fall back to others
order = ["Table 1", "Table 2", "Table 3", "Contents"]
df0, used = None, None
for s in order:
    if s in xl.sheet_names:
        df0, used = _read_tidy_from_sheet(xl, s)
        if df0 is not None:
            break

assert df0 is not None, f"Could not find a tidy table in {xl.sheet_names}. If this persists, re-download the ABS workbook."
print("Chosen sheet:", used)
print("Columns:", list(df0.columns))
df0.head()


Using workbook: abs_population_people_2011_2024.xlsx
Sheets: ['Contents', 'Table 1', 'Table 2', 'Table 3']


AssertionError: Could not find a tidy table in ['Contents', 'Table 1', 'Table 2', 'Table 3']. If this persists, re-download the ABS workbook.

In [7]:
from pathlib import Path
import pandas as pd

abs_path = Path("../data/raw/ABS")
xls_path = next(abs_path.glob("abs_population_people_2011_2024.xlsx"))
print("Using workbook:", xls_path.name)

# Open the workbook and list sheet names
xl = pd.ExcelFile(xls_path, engine="openpyxl")
print("Sheets:", xl.sheet_names)

# Peek at the top 30 rows of “Table 3”
raw = pd.read_excel(xl, sheet_name="Table 3", header=None)
raw.head(30)


Using workbook: abs_population_people_2011_2024.xlsx
Sheets: ['Contents', 'Table 1', 'Table 2', 'Table 3']


,0,1,2,3,4,5,6,7,8,9,...,114,115,116,117,118,119,120,121,122,123
0,Australian Bureau of Statistics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Data by region, 2011-24",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Released at 11.30am (Canberra time) 27 May 2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Table 3 POPULATION AND PEOPLE, Remoteness Area...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,Estimated resident population - year ended 30 ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Births and deaths - year ended 31 December,NaN,NaN,NaN
6,Code,Label,Year,Estimated resident population (no.),Population density (persons/km2),Estimated resident population - males (no.),Estimated resident population - females (no.),Median age - males (years),Median age - females (years),Median age - persons (years),...,Persons - 60-64 years (%),Persons - 65-69 years (%),Persons - 70-74 years (%),Persons - 75-79 years (%),Persons - 80-84 years (%),Persons - 85 and over (%),Births (no.),Total fertility rate (births per female) (rate),Deaths (no.),"Standardised death rate (per 1,000 people) (rate)"
7,10,Major Cities of Australia,2018,-,-,-,-,-,-,-,...,-,-,-,-,-,-,83830,1.72,36058,5.1
8,10,Major Cities of Australia,2019,6088337,1102.6,3024887,3063450,35.6,37.3,36.5,...,5.1,4.4,3.8,2.6,1.9,2,76939,1.7,37491,5.1
9,10,Major Cities of Australia,2020,6137535,1111.5,3048270,3089265,36,37.7,36.9,...,5.2,4.5,3.9,2.7,1.9,2,73972,1.68,35199,0.5


In [8]:
import re
import numpy as np
import pandas as pd

# Auto-detect a sensible header row: first row in Table 3 that mentions "SA2"
raw = pd.read_excel(xl, sheet_name="Table 3", header=None)
hdr_idx = raw.index[ raw.apply(lambda r: r.astype(str).str.contains("SA2", case=False, na=False).any(), axis=1) ][0]
print("Detected header row:", hdr_idx)

# Re-read with that header
df = pd.read_excel(xl, sheet_name="Table 3", header=hdr_idx)

# Drop rows that are completely empty in the first 2 columns (name/code)
df = df.dropna(how="all", axis=0).reset_index(drop=True)

# Find the "name" and "code" columns (be robust to slight wording)
def find_col(candidates, fallback=None):
    for c in df.columns:
        s = str(c).lower()
        if any(k in s for k in candidates):
            return c
    return fallback

name_col = find_col(["sa2", "name"])
code_col = find_col(["code", "asgs code", "sa2 code"])

assert name_col is not None, "Could not find the SA2 name column"
assert code_col is not None, "Could not find the SA2 code column"
print("Detected columns -> name:", name_col, "| code:", code_col)

# Keep only rows that look like real SA2s (filter out Totals/Notes)
keep = df[name_col].astype(str).str.contains(r"^\s*(?!total|note|source)", flags=re.I, regex=True, na=False)
df = df.loc[keep].copy()

# Identify year columns: strings that are exactly 4 digits (2011..2024)
year_cols = [c for c in df.columns if re.fullmatch(r"\d{4}", str(c))]
assert year_cols, "No year columns (e.g., 2011..2024) were found"

# Melt to tidy format: one row per SA2 per year
tidy = df.melt(
    id_vars=[code_col, name_col],
    value_vars=year_cols,
    var_name="year",
    value_name="SA2_POP"
)

# Clean types
tidy["year"] = pd.to_numeric(tidy["year"], errors="coerce").astype("Int64")
tidy["SA2_POP"] = pd.to_numeric(tidy["SA2_POP"], errors="coerce")

# Keep the latest year available
latest_year = int(tidy["year"].dropna().max())
pop_latest = tidy.loc[tidy["year"].eq(latest_year)].copy()

# Standardise column names for join
pop_latest = pop_latest.rename(columns={
    name_col: "SA2_NAME",
    code_col: "SA2_CODE_2021"
})[["SA2_CODE_2021", "SA2_NAME", "SA2_POP"]]

print("Latest year detected:", latest_year)
pop_latest.head()


IndexError: index 0 is out of bounds for axis 0 with size 0

In [9]:
from pathlib import Path
import pandas as pd, re

abs_dir = Path("../data/raw/ABS")
xls_path = next(abs_dir.glob("*.xlsx"))
print("Using workbook:", xls_path.name)

xl = pd.ExcelFile(xls_path, engine="openpyxl")
print("Sheets:", xl.sheet_names)

def read_abs_wide_table(xl, sheets=("Table 1","Table 2","Table 3")):
    """
    Find the header row by looking for a row that contains many year labels (2011..2024)
    and mentions 'sa2' or ('name' and 'code'). Works across Table 1/2/3.
    Returns (sheet_name, df) or (None, None).
    """
    for s in sheets:
        if s not in xl.sheet_names:
            continue
        raw = pd.read_excel(xl, sheet_name=s, header=None, dtype=str)
        # scan top rows for the header line
        header_idx = None
        for i in range(min(80, len(raw))):
            row = [("" if v is None else str(v).strip()) for v in raw.iloc[i].tolist()]
            low = [c.lower() for c in row]
            # how many cells look like year labels?
            yrs = sum(bool(re.fullmatch(r"20\d{2}", c)) for c in row)
            has_sa2  = any("sa2" in c for c in low)
            has_name = any("name" in c for c in low)
            has_code = any("code" in c for c in low)
            if yrs >= 5 and (has_sa2 or (has_name and has_code)):
                header_idx = i
                break
        if header_idx is None:
            # try a looser rule: lots of years is enough
            for i in range(min(80, len(raw))):
                row = [("" if v is None else str(v).strip()) for v in raw.iloc[i].tolist()]
                yrs = sum(bool(re.fullmatch(r"20\d{2}", c)) for c in row)
                if yrs >= 6:
                    header_idx = i
                    break
        if header_idx is None:
            continue

        df = pd.read_excel(xl, sheet_name=s, header=header_idx)
        # drop all-empty columns/rows
        df = df.dropna(how="all", axis=1)
        df = df.dropna(how="all", axis=0).reset_index(drop=True)
        return s, df

    return None, None

sheet, df = read_abs_wide_table(xl)
assert df is not None, f"Could not auto-detect the header on {xl.sheet_names}. As a fallback, run: pd.read_excel(xl,'Table 3',header=None).head(40)"

print("Chosen sheet:", sheet)
print("Columns (first 12):", list(df.columns)[:12])
df.head()


Using workbook: abs_population_people_2011_2024.xlsx
Sheets: ['Contents', 'Table 1', 'Table 2', 'Table 3']


AssertionError: Could not auto-detect the header on ['Contents', 'Table 1', 'Table 2', 'Table 3']. As a fallback, run: pd.read_excel(xl,'Table 3',header=None).head(40)

In [10]:
pd.read_excel(xl, sheet_name="Table 1", header=None).head(40)
pd.read_excel(xl, sheet_name="Table 2", header=None).head(40)
pd.read_excel(xl, sheet_name="Table 3", header=None).head(40)


,0,1,2,3,4,5,6,7,8,9,...,114,115,116,117,118,119,120,121,122,123
0,Australian Bureau of Statistics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Data by region, 2011-24",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Released at 11.30am (Canberra time) 27 May 2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Table 3 POPULATION AND PEOPLE, Remoteness Area...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,Estimated resident population - year ended 30 ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Births and deaths - year ended 31 December,NaN,NaN,NaN
6,Code,Label,Year,Estimated resident population (no.),Population density (persons/km2),Estimated resident population - males (no.),Estimated resident population - females (no.),Median age - males (years),Median age - females (years),Median age - persons (years),...,Persons - 60-64 years (%),Persons - 65-69 years (%),Persons - 70-74 years (%),Persons - 75-79 years (%),Persons - 80-84 years (%),Persons - 85 and over (%),Births (no.),Total fertility rate (births per female) (rate),Deaths (no.),"Standardised death rate (per 1,000 people) (rate)"
7,10,Major Cities of Australia,2018,-,-,-,-,-,-,-,...,-,-,-,-,-,-,83830,1.72,36058,5.1
8,10,Major Cities of Australia,2019,6088337,1102.6,3024887,3063450,35.6,37.3,36.5,...,5.1,4.4,3.8,2.6,1.9,2,76939,1.7,37491,5.1
9,10,Major Cities of Australia,2020,6137535,1111.5,3048270,3089265,36,37.7,36.9,...,5.2,4.5,3.9,2.7,1.9,2,73972,1.68,35199,0.5


In [11]:
import numpy as np

def find_name_col(df):
    # Prefer a column that has both 'sa2' and 'name' in its header
    for c in df.columns:
        s = str(c).lower()
        if "sa2" in s and "name" in s:
            return c
    # fallback: any 'name'
    for c in df.columns:
        if "name" in str(c).lower():
            return c
    return None

def find_code_col(df):
    for c in df.columns:
        s = str(c).lower()
        if "sa2" in s and "code" in s:
            return c
    for c in df.columns:
        if "code" in str(c).lower():
            return c
    return None

name_col = find_name_col(df)
code_col = find_code_col(df)
assert name_col is not None, "Could not find an SA2 'name' column"
assert code_col is not None, "Could not find an SA2 'code' column"
print("Detected -> name:", name_col, "| code:", code_col)

# Identify year columns (either string '2011'..'2024' or numeric)
year_cols = [c for c in df.columns if re.fullmatch(r"20\d{2}", str(c).strip())]
if not year_cols:
    year_cols = [c for c in df.columns if isinstance(c,(int,float)) and 2010 < int(c) < 2030]
assert year_cols, "No year columns found"

# Remove obvious totals/notes rows
mask_valid = df[name_col].astype(str).str.fullmatch(r"\s*(?!(?i)total|notes?|source).+", na=False)
dfw = df.loc[mask_valid].copy()

# Melt wide -> long tidy
tidy = dfw.melt(
    id_vars=[code_col, name_col],
    value_vars=year_cols,
    var_name="year",
    value_name="SA2_POP"
)

# Clean types
tidy["year"] = pd.to_numeric(tidy["year"], errors="coerce").astype("Int64")
tidy["SA2_POP"] = pd.to_numeric(tidy["SA2_POP"], errors="coerce")

latest_year = int(tidy["year"].dropna().max())
pop_latest = tidy.loc[tidy["year"].eq(latest_year)].copy()

pop_latest = pop_latest.rename(columns={
    name_col: "SA2_NAME",
    code_col: "SA2_CODE_2021"
})[["SA2_CODE_2021","SA2_NAME","SA2_POP"]]

print("Latest year detected:", latest_year, "| rows:", len(pop_latest))
pop_latest.head()


AttributeError: 'NoneType' object has no attribute 'columns'

In [12]:
# --- Load SA2 ERP (population) and join to sensors ---

import pandas as pd
from pathlib import Path

RAW = Path("../data/raw")
INTERIM = Path("../data/interim")

# 1) Read the ERP file you saved in Step 1
erp_path = RAW / "ABS" / "erp_sa2_2023.csv"
assert erp_path.exists(), f"Missing file: {erp_path}. Please download and save it first."

erp = pd.read_csv(erp_path)

# 2) Normalise column names (handle both Atlas and Data-by-Region shapes)
cols = {c: c.strip().upper() for c in erp.columns}
erp.columns = cols.keys()

# Atlas export has columns like: SA2_CODE_2021, SA2_NAME_2021, ERP, Year
# Data-by-Region CSV has: SA2_CODE_2021, SA2_NAME_2021, YEAR, INDICATOR, VALUE
code_col  = next((c for c in erp.columns if c.upper().startswith("SA2_CODE")), None)
name_col  = next((c for c in erp.columns if c.upper().startswith("SA2_NAME")), None)
year_col  = next((c for c in erp.columns if c.upper()=="YEAR"), None)

# pick the population value column
if "ERP" in erp.columns:
    pop_col = "ERP"
elif "VALUE" in erp.columns:
    # filter to Indicator == Estimated resident population if present
    ind_col = next((c for c in erp.columns if c.upper()=="INDICATOR"), None)
    if ind_col and "Estimated resident population" in set(erp[ind_col].astype(str)):
        erp = erp[erp[ind_col].astype(str).str.contains("Estimated resident population", case=False, na=False)]
    pop_col = "VALUE"
else:
    raise AssertionError("Could not find a population value column (ERP or VALUE).")

assert code_col and name_col and year_col, "Could not find SA2 code/name/year columns."

# 3) Keep latest year only (e.g., 2023)
latest_year = int(pd.to_numeric(erp[year_col], errors="coerce").max())
erp_latest = erp[pd.to_numeric(erp[year_col], errors="coerce").eq(latest_year)].copy()

# tidy types and names
erp_latest["SA2_CODE_2021"] = erp_latest[code_col].astype(str).str.strip()
erp_latest["SA2_NAME"]      = erp_latest[name_col].astype(str).str.strip()
erp_latest["SA2_POP"]       = pd.to_numeric(erp_latest[pop_col], errors="coerce")

erp_tidy = erp_latest[["SA2_CODE_2021", "SA2_NAME", "SA2_POP"]].dropna(subset=["SA2_CODE_2021"])

print(f"ERP rows (latest={latest_year}):", len(erp_tidy))
erp_tidy.head()


AssertionError: Could not find a population value column (ERP or VALUE).

In [13]:
from pathlib import Path
import pandas as pd
import re

RAW = Path("../data/raw")
erp_path = RAW / "ABS" / "erp_sa2_2023.csv"   # <- change if your file has a different name
assert erp_path.exists(), f"Missing file: {erp_path}. Please download and save it first."

erp = pd.read_csv(erp_path)
print("Rows x Cols:", erp.shape)
print("First 20 columns:", list(erp.columns)[:20])
erp.head(3)


Rows x Cols: (2473, 31)
First 20 columns: ['OBJECTID', 'Statistical Areas Level 2 2021 code', 'Statistical Areas Level 2 2021 name', 'Area in square kilometres', 'Uniform Resource Identifier', '2001 Estimated resident population (no.)', '2002 Estimated resident population (no.)', '2003 Estimated resident population (no.)', '2004 Estimated resident population (no.)', '2005 Estimated resident population (no.)', '2006 Estimated resident population (no.)', '2007 Estimated resident population (no.)', '2008 Estimated resident population (no.)', '2009 Estimated resident population (no.)', '2010 Estimated resident population (no.)', '2011 Estimated resident population (no.)', '2012 Estimated resident population (no.)', '2013 Estimated resident population (no.)', '2014 Estimated resident population (no.)', '2015 Estimated resident population (no.)']


,OBJECTID,Statistical Areas Level 2 2021 code,Statistical Areas Level 2 2021 name,Area in square kilometres,Uniform Resource Identifier,2001 Estimated resident population (no.),2002 Estimated resident population (no.),2003 Estimated resident population (no.),2004 Estimated resident population (no.),2005 Estimated resident population (no.),...,2017 Estimated resident population (no.),2018 Estimated resident population (no.),2019 Estimated resident population (no.),2020 Estimated resident population (no.),2021 Estimated resident population (no.),2022 Estimated resident population (no.),2023 Estimated resident population (no.),globalid,Shape__Area,Shape__Length
0,1,101021007,Braidwood,3418.3525,https://linked.data.gov.au/dataset/asgsed3/SA2...,2760.0,2811.0,2835.0,2844.0,2847.0,...,4041.0,4145.0,4218.0,4282.0,4332.0,4366.0,4396.0,d6182ac8-efdb-4293-b59c-0119eb98b3d7,0.339397,3.913694
1,2,101021008,Karabar,6.9825,https://linked.data.gov.au/dataset/asgsed3/SA2...,9129.0,9199.0,9263.0,9277.0,9209.0,...,8530.0,8516.0,8500.0,8535.0,8548.0,8528.0,8483.0,c92e2601-9221-47f7-9453-3bc19567fe67,0.000693,0.138642
2,3,101021009,Queanbeyan,4.7620,https://linked.data.gov.au/dataset/asgsed3/SA2...,9717.0,9513.0,9522.0,9400.0,9595.0,...,11362.0,11460.0,11468.0,11460.0,11375.0,11391.0,11420.0,95e5e156-f9a0-4c16-a2fb-ecb5ef2ad685,0.000472,0.107150


In [14]:
# Make a case-insensitive lookup by working with an upper-cased version of each column name
upper = {c.upper(): c for c in erp.columns}

# Find the SA2 code and name columns (robust to slight variations)
code_col = next((upper[k] for k in upper if k.startswith("SA2_CODE")), None)
name_col = next((upper[k] for k in upper if k.startswith("SA2_NAME")), None)

assert code_col and name_col, f"Could not find SA2 code/name columns. Saw: {list(erp.columns)[:10]}"

print("Detected:", code_col, "|", name_col)

# Find all ERP year columns like ERP_2001, ERP_2002, ..., ERP_2023
erp_year_cols = []
for c in erp.columns:
    m = re.search(r'(?:^ERP[_ ]?)(20\d{2})$', c.strip(), flags=re.I)
    if m:
        year = int(m.group(1))
        erp_year_cols.append((c, year))

# Safety: sometimes columns can be just '2001', '2002', ... (rare, but handle it)
if not erp_year_cols:
    for c in erp.columns:
        m = re.fullmatch(r'(20\d{2})', c.strip())
        if m:
            year = int(m.group(1))
            erp_year_cols.append((c, year))

# Final checks
years_found = sorted({y for _, y in erp_year_cols})
assert erp_year_cols, "Couldn't find any ERP_* year columns. Inspect erp.head() to see real column names."

print("Years found:", years_found[:10], " ... last:", years_found[-1])


AssertionError: Could not find SA2 code/name columns. Saw: ['OBJECTID', 'Statistical Areas Level 2 2021 code', 'Statistical Areas Level 2 2021 name', 'Area in square kilometres', 'Uniform Resource Identifier', '2001 Estimated resident population (no.)', '2002 Estimated resident population (no.)', '2003 Estimated resident population (no.)', '2004 Estimated resident population (no.)', '2005 Estimated resident population (no.)']

In [15]:
from pathlib import Path
import pandas as pd
import re

RAW = Path("../data/raw")
erp_path = RAW / "ABS" / "erp_sa2_2023.csv"     # change if your file name is different
assert erp_path.exists(), f"Missing file: {erp_path}"

erp = pd.read_csv(erp_path)
print("Rows x Cols:", erp.shape)
print("First 20 columns:", list(erp.columns)[:20])
erp.head(3)


Rows x Cols: (2473, 31)
First 20 columns: ['OBJECTID', 'Statistical Areas Level 2 2021 code', 'Statistical Areas Level 2 2021 name', 'Area in square kilometres', 'Uniform Resource Identifier', '2001 Estimated resident population (no.)', '2002 Estimated resident population (no.)', '2003 Estimated resident population (no.)', '2004 Estimated resident population (no.)', '2005 Estimated resident population (no.)', '2006 Estimated resident population (no.)', '2007 Estimated resident population (no.)', '2008 Estimated resident population (no.)', '2009 Estimated resident population (no.)', '2010 Estimated resident population (no.)', '2011 Estimated resident population (no.)', '2012 Estimated resident population (no.)', '2013 Estimated resident population (no.)', '2014 Estimated resident population (no.)', '2015 Estimated resident population (no.)']


,OBJECTID,Statistical Areas Level 2 2021 code,Statistical Areas Level 2 2021 name,Area in square kilometres,Uniform Resource Identifier,2001 Estimated resident population (no.),2002 Estimated resident population (no.),2003 Estimated resident population (no.),2004 Estimated resident population (no.),2005 Estimated resident population (no.),...,2017 Estimated resident population (no.),2018 Estimated resident population (no.),2019 Estimated resident population (no.),2020 Estimated resident population (no.),2021 Estimated resident population (no.),2022 Estimated resident population (no.),2023 Estimated resident population (no.),globalid,Shape__Area,Shape__Length
0,1,101021007,Braidwood,3418.3525,https://linked.data.gov.au/dataset/asgsed3/SA2...,2760.0,2811.0,2835.0,2844.0,2847.0,...,4041.0,4145.0,4218.0,4282.0,4332.0,4366.0,4396.0,d6182ac8-efdb-4293-b59c-0119eb98b3d7,0.339397,3.913694
1,2,101021008,Karabar,6.9825,https://linked.data.gov.au/dataset/asgsed3/SA2...,9129.0,9199.0,9263.0,9277.0,9209.0,...,8530.0,8516.0,8500.0,8535.0,8548.0,8528.0,8483.0,c92e2601-9221-47f7-9453-3bc19567fe67,0.000693,0.138642
2,3,101021009,Queanbeyan,4.7620,https://linked.data.gov.au/dataset/asgsed3/SA2...,9717.0,9513.0,9522.0,9400.0,9595.0,...,11362.0,11460.0,11468.0,11460.0,11375.0,11391.0,11420.0,95e5e156-f9a0-4c16-a2fb-ecb5ef2ad685,0.000472,0.107150


In [16]:
# Make an UPPERCASE lookup so we can match case-insensitively but still get the real column back
upper = {c.upper(): c for c in erp.columns}

# 2a) Find the SA2 code & name columns
code_candidates = [upper[k] for k in upper if re.search(r'(SA2|STATISTICAL AREAS LEVEL 2).*CODE', k, re.I)]
name_candidates = [upper[k] for k in upper if re.search(r'(SA2|STATISTICAL AREAS LEVEL 2).*NAME', k, re.I)]

code_col = code_candidates[0] if code_candidates else None
name_col = name_candidates[0] if name_candidates else None

assert code_col and name_col, (
    "Could not find SA2 code/name columns.\n"
    f"Saw (first 12): {list(erp.columns)[:12]}"
)

print("Detected code/name:", code_col, "|", name_col)

# 2b) Find ALL year columns like '2001 Estimated resident population (no.)'
year_cols = []   # list of (col_name, year_int)

for c in erp.columns:
    m = re.match(r'^\s*(20\d{2})\b.*ESTIMATED\s+RESIDENT\s+POPULATION', c, flags=re.I)
    if m:
        year_cols.append((c, int(m.group(1))))

# Fallback (rare): if those didn't match, try columns that simply start with a year
if not year_cols:
    for c in erp.columns:
        m = re.match(r'^\s*(20\d{2})\b', c)
        if m:
            year_cols.append((c, int(m.group(1))))

assert year_cols, (
    "Couldn't find any year population columns. "
    "Look at erp.head() to see the actual headings."
)

years_found = sorted({y for _, y in year_cols})
print("Years found (first few):", years_found[:10], "| last:", years_found[-1])


Detected code/name: Statistical Areas Level 2 2021 code | Statistical Areas Level 2 2021 name
Years found (first few): [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010] | last: 2023


In [17]:
# Melt to tidy (long) with columns: code, name, year, SA2_POP
id_vars = [code_col, name_col]
value_vars = [c for c, _ in year_cols]

tidy = erp.melt(id_vars=id_vars, value_vars=value_vars,
                var_name="source_col", value_name="SA2_POP")

# Extract the year from the column header
tidy["year"] = tidy["source_col"].str.extract(r'(20\d{2})').astype(int)

latest_year = tidy["year"].max()
erp_latest = tidy.loc[tidy["year"] == latest_year, [code_col, name_col, "SA2_POP"]].copy()

# Clean up types/whitespace
erp_latest[code_col] = erp_latest[code_col].astype(str).str.strip()
erp_latest[name_col] = erp_latest[name_col].astype(str).str.strip()
erp_latest["SA2_POP"] = pd.to_numeric(erp_latest["SA2_POP"], errors="coerce")

print("Latest year detected:", latest_year, "| rows:", len(erp_latest))
erp_latest.head()


Latest year detected: 2023 | rows: 2473


,Statistical Areas Level 2 2021 code,Statistical Areas Level 2 2021 name,SA2_POP
54406,101021007,Braidwood,4396.0
54407,101021008,Karabar,8483.0
54408,101021009,Queanbeyan,11420.0
54409,101021010,Queanbeyan - East,5099.0
54410,101021012,Queanbeyan West - Jerrabomberra,12873.0


In [18]:
INTERIM = Path("../data/interim")
INTERIM.mkdir(parents=True, exist_ok=True)

out = erp_latest.rename(columns={
    code_col: "SA2_CODE_2021",
    name_col: "SA2_NAME"
})[["SA2_CODE_2021", "SA2_NAME", "SA2_POP"]].dropna(subset=["SA2_POP"])

out_path = INTERIM / "erp_sa2_population_latest.csv"
out.to_csv(out_path, index=False)
print("Wrote:", out_path, "| Rows:", len(out))


Wrote: ../data/interim/erp_sa2_population_latest.csv | Rows: 2454


In [20]:
!git add data/interim/erp_sa2_population_latest.csv data/interim/sensor_sa2_population.csv
!git commit -m "ABS ERP SA2 latest (Atlas CSV with long labels); joined to sensors"
!git push -u origin main


fatal: pathspec 'data/interim/erp_sa2_population_latest.csv' did not match any files
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   02_clean_join_geo.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../Miniforge3-MacOSX-arm64.sh

no changes added to commit (use "git add" and/or "git commit -a")
branch 'main' set up to track 'origin/main'.
Everything up-to-date


In [22]:
from pathlib import Path
import re
import pandas as pd

RAW = Path("../data/raw/ABS")
erp_path = RAW / "erp_sa2_2023.csv"   # use the file you downloaded
assert erp_path.exists(), f"Missing: {erp_path}"

erp = pd.read_csv(erp_path)

# ---- Normalise column names for flexible matching
upper = {c: c.strip().upper() for c in erp.columns}

# SA2 code & name: accept variants like “SA2_CODE_2021” or “Statistical Areas Level 2 2021 code”
def find_first(cols, *needles):
    for c in erp.columns:
        u = upper[c]
        if all(n in u for n in needles):
            return c
    return None

code_col = find_first(erp.columns, "SA2", "CODE") or find_first(erp.columns, "STATISTICAL AREAS LEVEL 2", "CODE")
name_col = find_first(erp.columns, "SA2", "NAME") or find_first(erp.columns, "STATISTICAL AREAS LEVEL 2", "NAME")
year_col = find_first(erp.columns, "YEAR")  # sometimes present
assert code_col and name_col, f"Could not find SA2 code/name columns. Saw: {list(erp.columns)[:10]}"

# ---- Collect all ERP year columns like ERP_2001, 2002, ..., or “2001 Estimated resident population (no.)”
year_cols = []
for c in erp.columns:
    u = upper[c]
    m = re.search(r'(?:^ERP_)?(20\d{2})\b', u)  # matches ERP_2023 or 2023
    if m:
        year_cols.append((int(m.group(1)), c))

# fallback: plain “2001”, “2002”, … columns
if not year_cols:
    for c in erp.columns:
        m = re.fullmatch(r'\s*(20\d{2})\s*', str(c).strip())
        if m: year_cols.append((int(m.group(1)), c))

assert year_cols, "No ERP year columns found. Inspect erp.head() to see real headings."

latest_year, latest_col = sorted(year_cols)[-1]

# ---- Tidy to one row per SA2 for latest year
erp_latest = erp[[code_col, name_col, latest_col]].copy()
erp_latest.columns = ["SA2_CODE_2021", "SA2_NAME", "SA2_POP"]
erp_latest["SA2_CODE_2021"] = erp_latest["SA2_CODE_2021"].astype(str).str.strip().str.zfill(9)
erp_latest["SA2_NAME"] = erp_latest["SA2_NAME"].astype(str).str.strip()
erp_latest["SA2_POP"] = pd.to_numeric(erp_latest["SA2_POP"], errors="coerce")

print(f"Latest year detected: {latest_year} | rows:", len(erp_latest))
erp_latest.head()


Latest year detected: 2023 | rows: 2473


,SA2_CODE_2021,SA2_NAME,SA2_POP
0,101021007,Braidwood,4396.0
1,101021008,Karabar,8483.0
2,101021009,Queanbeyan,11420.0
3,101021010,Queanbeyan - East,5099.0
4,101021012,Queanbeyan West - Jerrabomberra,12873.0


In [23]:
assert erp_latest["SA2_CODE_2021"].str.len().eq(9).all(), "SA2 codes should be 9 digits"
assert erp_latest["SA2_NAME"].notna().all(), "Missing SA2 names"
print("Unique SA2s:", erp_latest["SA2_CODE_2021"].nunique())


Unique SA2s: 2473


In [24]:
sensor_sa2 = pd.read_csv("../data/interim/sensor_sa2.csv")
# normalise names to reduce tiny formatting mismatches (hyphens/extra spaces)
def norm_name(s):
    return (s.astype(str)
              .str.replace(r"[–—\-]", "-", regex=True)   # unify dashes
              .str.replace(r"\s+", " ", regex=True)
              .str.strip()
            )

sensor_sa2["SA2_NAME_N"] = norm_name(sensor_sa2["SA2_NAME"])
erp_latest["SA2_NAME_N"] = norm_name(erp_latest["SA2_NAME"])

joined = (sensor_sa2
          .merge(erp_latest[["SA2_NAME_N","SA2_POP","SA2_CODE_2021"]],
                 on="SA2_NAME_N", how="left")
          .drop(columns=["SA2_NAME_N"]))

coverage = joined["SA2_POP"].notna().mean()*100
print(f"Sensor→SA2 population coverage: {coverage:.1f}%")
missing = joined[joined["SA2_POP"].isna()][["sensor_id","sensor_name","SA2_NAME"]].head(10)
if not missing.empty:
    print("Examples needing name tidy/manually check:", missing.to_dict("records")[:5])


Sensor→SA2 population coverage: 100.0%


In [25]:
from pathlib import Path
INTERIM = Path("../data/interim"); INTERIM.mkdir(parents=True, exist_ok=True)

erp_out   = INTERIM / "erp_sa2_population_latest.csv"
joined_out= INTERIM / "sensor_sa2_population.csv"

erp_latest.to_csv(erp_out, index=False)
joined.to_csv(joined_out, index=False)

print("Wrote:", erp_out, "| Rows:", len(erp_latest))
print("Wrote:", joined_out, "| Rows:", len(joined))


Wrote: ../data/interim/erp_sa2_population_latest.csv | Rows: 2473
Wrote: ../data/interim/sensor_sa2_population.csv | Rows: 135
